In [33]:
import tensorflow as tf
import numpy as np

Vi starter med at loaded den givne data ind. Her er dataen sepereret i lyd med musik og lyd uden musik. Det skal vi så sætte sammen i et array og lave en array som indeholder om hvilken type hver styk data er. Her kan vi bruge numpy siden hver fil er i .npy format

In [34]:
music_data = np.load("music_data.npy")
other_data = np.load("other_data.npy")
test_data = np.load("test_data.npy")

Data layoutet er her $10500$ 2d arrays af $30\times79$ som repræsentere et log-mel spektogram udregnet fra den givne lyd sample. $x$ dimensionen er frekvens båndene mens $y$ er tids rammen

Nu skal vi sætte `music_data` og `other_data` sammen til et array og lave en array som indeholder hvilken type hver lyd sample er. Vi vil også gerne have noget test data så vi kan evaluere hvor godt netværket er uden at overfitte Det gør vi ved at efterlade 1000 fra `music_dat` og `other_data`

In [35]:
music_train_data = music_data[0: -1000]
other_train_data = other_data[0: -1000]
train_data = (
	np.concatenate((music_train_data, other_train_data)),
	np.concatenate((
		np.full((len(music_train_data), 2), (1, 0)), 
		np.full((len(other_train_data), 2), (0, 1))
	))
)	

music_test_data = music_data[len(music_data) - 1000:None]
other_test_data = other_data[len(other_data) - 1000:None]
test_data = (
	np.concatenate((music_test_data, other_test_data)),
	np.concatenate((
		np.full((len(music_test_data), 2), (1, 0)),
		np.full((len(other_test_data), 2), (0, 1))
	))
)


Nu har vi lavet dataset til test og træning, nu skal vi sætte vores model op

In [36]:
model = tf.keras.models.Sequential([
	tf.keras.layers.Reshape((30, 79, 1), input_shape = (30, 79)),
	tf.keras.layers.Conv2D(3, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(32, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Conv2D(64, (3, 3), activation = "relu"),
	tf.keras.layers.MaxPool2D(),
	tf.keras.layers.Flatten(),
	#tf.keras.layers.Dense(1024, activation = "relu"),
	#tf.keras.layers.Dense(512, activation = "relu"),
	tf.keras.layers.Dense(2, activation = "softmax")
])

model.build(input_shape=(30, 79))

model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_3 (Reshape)         (None, 30, 79, 1)         0         
                                                                 
 conv2d_9 (Conv2D)           (None, 28, 77, 3)         30        
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 14, 38, 3)        0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 12, 36, 32)        896       
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 6, 18, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_11 (Conv2D)          (None, 4, 16, 64)        

Nu hvor vi har en model, så skal den bare trænes

In [37]:
predictions = model(train_data[0][:10]).numpy()
predictions

array([[7.4873106e-03, 9.9251270e-01],
       [3.5908768e-01, 6.4091235e-01],
       [4.1603707e-02, 9.5839632e-01],
       [7.9988167e-03, 9.9200118e-01],
       [2.8759198e-02, 9.7124082e-01],
       [5.1381388e-03, 9.9486190e-01],
       [1.7684403e-01, 8.2315600e-01],
       [5.7530687e-03, 9.9424696e-01],
       [1.9648224e-01, 8.0351776e-01],
       [1.5700408e-04, 9.9984300e-01]], dtype=float32)

Nu laver vi en loss funktion, siden vi kun har et output så bruger vi binarycrossentropy

In [38]:
model.compile(optimizer='adam', loss='CategoricalCrossentropy', metrics=['accuracy'])

Så træner vi modellen med dataen over 10 epochs

In [39]:
history = model.fit(train_data[0], train_data[1], epochs=10)

Epoch 1/10
594/594 [==============================] - 9s 15ms/step - loss: 0.4933 - accuracy: 0.8043
Epoch 2/10
594/594 [==============================] - 9s 16ms/step - loss: 0.1924 - accuracy: 0.9220
Epoch 3/10
594/594 [==============================] - 9s 14ms/step - loss: 0.1441 - accuracy: 0.9462
Epoch 4/10
594/594 [==============================] - 9s 15ms/step - loss: 0.1084 - accuracy: 0.9603
Epoch 5/10
594/594 [==============================] - 9s 15ms/step - loss: 0.0863 - accuracy: 0.9697
Epoch 6/10
594/594 [==============================] - 9s 16ms/step - loss: 0.0750 - accuracy: 0.9725
Epoch 7/10
594/594 [==============================] - 9s 15ms/step - loss: 0.0648 - accuracy: 0.9770
Epoch 8/10
594/594 [==============================] - 9s 15ms/step - loss: 0.0527 - accuracy: 0.9815
Epoch 9/10
594/594 [==============================] - 9s 15ms/step - loss: 0.0524 - accuracy: 0.9811
Epoch 10/10
594/594 [==============================] - 9s 15ms/step - loss: 0.0451 - accura

In [40]:
train_evaluation = model.evaluate(train_data[0], train_data[1])
test_evaluation = model.evaluate(test_data[0], test_data[1])
print("Training set accuracy %.3f%% and loss: %.3f%%" % 
	(train_evaluation[1] * 100, train_evaluation[0] * 100))
print("Test set accuracy %.3f%% and loss: %.3f%%" % 
	(test_evaluation[1] * 100, test_evaluation[0] * 100))

63/63 [==============================] - 0s 7ms/step - loss: 0.0563 - accuracy: 0.9810
Training set accuracy 98.884% and loss: 3.281%
Test set accuracy 98.100% and loss: 5.635%
